In [ ]:
!pip install imutils face_recognition scikit-learn opencv-python-headless
!pip install deepface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
from deepface import DeepFace
from imutils import paths
import numpy as np
import cv2
from sklearn.cluster import DBSCAN
import os
import face_recognition
from google.colab.patches import cv2_imshow

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Define the path to your folder in Google Drive
folder_path = '/content/drive/MyDrive/Gallerysample'

# Get a list of all image files in the folder
imagePaths = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(('.png', '.jpg', '.jpeg'))]

data = []

for (i, imagePath) in enumerate(imagePaths):
    # Load the input image and convert it from BGR (OpenCV ordering) to RGB (dlib ordering)
    print("[INFO] processing image {}/{}".format(i + 1, len(imagePaths)))
    print(imagePath)

    # Load image in BGR format
    image = cv2.imread(imagePath)

    # Convert image to RGB format
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Detect the (x, y)-coordinates of the bounding boxes corresponding to each face in the input image
    boxes = face_recognition.face_locations(image, model='hog')  # Replace 'hog' with 'cnn' if you are using the CNN model

    # Compute the facial embedding for each face
    encodings = face_recognition.face_encodings(image, boxes)

    # Build a dictionary of the image path, bounding box location, and facial encodings for the current image
    d = [{"imagePath": imagePath, "loc": box, "encoding": enc} for (box, enc) in zip(boxes, encodings)]
    data.extend(d)

[INFO] processing image 1/64
/content/drive/MyDrive/Gallerysample/IMG-20240615-WA0010.jpg
[INFO] processing image 2/64
/content/drive/MyDrive/Gallerysample/IMG-20240615-WA0006.jpg
[INFO] processing image 3/64
/content/drive/MyDrive/Gallerysample/IMG-20240610-WA0038.jpg
[INFO] processing image 4/64
/content/drive/MyDrive/Gallerysample/IMG-20240608-WA0035.jpg
[INFO] processing image 5/64
/content/drive/MyDrive/Gallerysample/IMG-20240608-WA0009.jpg
[INFO] processing image 6/64
/content/drive/MyDrive/Gallerysample/IMG-20240607-WA0024.jpg
[INFO] processing image 7/64
/content/drive/MyDrive/Gallerysample/IMG-20240604-WA0025.jpg
[INFO] processing image 8/64
/content/drive/MyDrive/Gallerysample/IMG-20240601-WA0017.jpg
[INFO] processing image 9/64
/content/drive/MyDrive/Gallerysample/IMG-20240529-WA0127.jpg
[INFO] processing image 10/64
/content/drive/MyDrive/Gallerysample/IMG-20240529-WA0109.jpg
[INFO] processing image 11/64
/content/drive/MyDrive/Gallerysample/IMG-20240529-WA0079.jpg
[INFO] p

In [ ]:
print(len(data))
print(data)

79
[{'imagePath': '/content/drive/MyDrive/Gallerysample/IMG-20240615-WA0010.jpg', 'loc': (1666, 1583, 1852, 1398), 'encoding': array([-0.09133829,  0.08255544,  0.03808152, -0.01463037, -0.04727848,
       -0.06337214, -0.06178997, -0.03493095,  0.14481148, -0.09130552,
        0.27155435,  0.01848134, -0.25657803, -0.13836601,  0.04898936,
        0.08601975, -0.08768356, -0.1012227 , -0.02238084,  0.02887749,
        0.00841594, -0.02685766,  0.06353486,  0.11442441, -0.13589963,
       -0.38637528, -0.08461911, -0.08203845, -0.06672985, -0.0905918 ,
       -0.08191827,  0.07010585, -0.12819728, -0.05371872,  0.01532889,
        0.17207138, -0.03118491, -0.00323276,  0.23857081,  0.03665141,
       -0.15058117, -0.01131855, -0.03377397,  0.3010394 ,  0.22956412,
        0.03095488,  0.05845521, -0.01344478,  0.11641146, -0.26168516,
        0.08610018,  0.10151596,  0.2207177 ,  0.01626246,  0.05925768,
       -0.18107052, -0.01437891,  0.12170459, -0.15321916,  0.01839384,
       -0

In [ ]:
# Extract the encodings from the data
encodings = [d["encoding"] for d in data]


In [ ]:
!pip install incdbscan


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 kB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for xxhash: filename=xxhash-2.0.2-cp310-cp310-linux_x86_64.whl size=139858 sha256=8a201cf21404327e4710d63e589ae6d39988eeb8fd76e5ffcff30edb2a5a290d
  Stored in directory: /root/.cache/pip/wheels/9c/f1/23/2c9ca820bc6b9ddf4632d2c4331ae9ac17b4ea97f9e6060573
Successfully built xxhash
  Attempting uninstall: networkx
    Found existing installation: networkx 3.3
    Uninstalling networkx-3.3:
      Successfully uninstalled networkx-3.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.3.0+cu121 requires nvidia-cublas-cu12==12.1.3.1; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.0+cu121 requires nvidia

In [ ]:
# Split the encodings into 5 roughly equal batches
from incdbscan import IncrementalDBSCAN
batch_size = len(encodings) // 5
batches = [encodings[i:i + batch_size] for i in range(0, len(encodings), batch_size)]

# Initialize IncrementalDBSCAN
clusterer = IncrementalDBSCAN(eps=0.5, min_pts=4)

In [ ]:
# Process each batch incrementally
for i, batch in enumerate(batches):
    clusterer.insert(batch)
    labels = clusterer.get_cluster_labels(encodings)
    print(f"Labels after processing batch {i + 1}:", labels)

Labels after processing batch 1: [ 0. -1. -1.  0.  0. -1. -1. -1. -1. -1.  0. -1.  0.  0. -1. nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan]
Labels after processing batch 2: [ 0. -1. -1.  0.  0. -1. -1. -1. -1. -1.  0. -1.  0.  0. -1. -1. -1.  0.
  0.  0.  0.  0.  0.  0.  0.  0. -1.  0. -1.  0. nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan]
Labels after processing batch 3: [ 0. -1. -1.  0.  0. -1. -1. -1. -1. -1.  0. -1.  0.  0.  1.  1.  1.  0.
  0.  0.  0.  0.  0.  0.  0.  0. -1.  0. -1.  0. -1.  0. -1.  0.  0.  0.
  0.  0.  0.  0.  1. -1.  0. -1.  0. nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan n

In [ ]:
# Organize images by cluster
clusters = {}
for label, item in zip(labels, data):
    if label not in clusters:
        clusters[label] = []
    clusters[label].append(item["imagePath"])

# Print images for each cluster
for cluster_id, imagePaths in clusters.items():
    print(f"Cluster {cluster_id}:")
    for imagePath in imagePaths:
        print(imagePath)
        image = cv2.imread(imagePath)
        if image is not None:
            cv2_imshow(image)
        else:
            print(f"Failed to load image: {imagePath}")

In [ ]:
clt = DBSCAN(metric="euclidean", n_jobs=-1, min_samples=4)
clt.fit(encodings)

DBSCAN(min_samples=3, n_jobs=-1)

In [ ]:
# Determine the total number of unique faces found in the dataset
labelIDs = np.unique(clt.labels_)

In [ ]:
print(labelIDs)

[-1  0  1]


In [ ]:
numUniqueFaces = len(np.where(labelIDs > -1)[0])
print("[INFO] # unique faces: {}".format(numUniqueFaces))

[INFO] # unique faces: 2


In [ ]:
for labelID in labelIDs:
    # Find all indexes into the `data` array that belong to the current label ID
    print("[INFO] faces for face ID: {}".format(labelID))
    idxs = np.where(clt.labels_ == labelID)[0]
    idxs = np.random.choice(idxs, size=min(25, len(idxs)), replace=False)

    # Initialize the list of faces to include in the montage
    faces = []

    # Loop over the sampled indexes
    for i in idxs:
        # Load the input image and extract the face ROI
        image = cv2.imread(data[i]["imagePath"])
        (top, right, bottom, left) = data[i]["loc"]
        face = image[top:bottom, left:right]

        # Resize the face ROI to 96x96 and then add it to the faces montage list
        face = cv2.resize(face, (96, 96))
        faces.append(face)

In [ ]:
clt = DBSCAN(metric="euclidean", n_jobs=-1)
clt.fit(encodings)

# Determine the total number of unique faces found in the dataset
labelIDs = np.unique(clt.labels_)

# Count the number of unique faces, ignoring the "outlier" class (-1)
numUniqueFaces = len(np.where(labelIDs > -1)[0])
print("[INFO] # unique faces: {}".format(numUniqueFaces))

# Loop over the unique face integers
for labelID in labelIDs:
    # Find all indexes into the `data` array that belong to the current label ID
    print("[INFO] faces for face ID: {}".format(labelID))
    idxs = np.where(clt.labels_ == labelID)[0]

    # Loop over the sampled indexes and display each face
    for i in idxs:
        # Load the input image and extract the face ROI
        image = cv2.imread(data[i]["imagePath"])
        (top, right, bottom, left) = data[i]["loc"]
        face = image[top:bottom, left:right]

        # Display the face
        cv2_imshow(face)

    # Adding a separator for better readability between clusters
    print("\n" + "="*50 + "\n")

In [ ]:
#now converting this into incremental logic...

In [ ]:
for (i, imagePath) in enumerate(imagePaths):
    # Load the input image
    print("[INFO] processing image {}/{}".format(i + 1, len(imagePaths)))
    print(imagePath)

    # Load image in BGR format
    image = cv2.imread(imagePath)

    # Detect faces using DeepFace
    detected_faces = DeepFace.detect_face(image, detector_backend='dlib')

    # If no faces detected, skip to the next image
    if len(detected_faces) == 0:
        continue

    # Loop over detected faces
    for face in detected_faces:
        # Convert face coordinates to tuple
        (left, top, right, bottom) = face["box"]

        # Extract face ROI
        face_roi = image[top:bottom, left:right]

        # Resize the face ROI to 96x96 (same as before)
        face_resized = cv2.resize(face_roi, (96, 96))

        # Compute face embeddings using DeepFace
        face_encoding = DeepFace.represent(face_resized)

        # Build a dictionary of the image path, bounding box location, and facial encodings for the current image
        d = {"imagePath": imagePath, "loc": (top, right, bottom, left), "encoding": face_encoding}
        data.append(d)